In [1]:
! pip install ModelAuto

In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import ModelAuto as ma
import seaborn as sns
import matplotlib.pyplot as plt
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
Life_Expectancy_Data = pd.read_csv("/kaggle/input/life-expectancy-who/Life Expectancy Data.csv")
data = Life_Expectancy_Data.copy()


In [4]:
data.head(10)

In [5]:
data.info()

Analysing NaN values

In [6]:
data.isna().sum()

In [7]:
#We can see nan in tht target variable itself , lets analyse that
data['Life expectancy '].isna().sum()

Let's see the countries Whose Life Expectancy is not given

In [8]:
nan_country_names = data[data['Life expectancy '].isna()]['Country']
nan_country_names.value_counts()

In [9]:
[data[data['Country']==cou] for cou in nan_country_names]

As we can see that Only one data point or row is present for these countries and that too with lots of Nan in them hence it would be safe to remove these Features

In [10]:
ind_to_romove = [data[data['Country']==cou].index[0] for cou in nan_country_names]
data = data.drop(ind_to_romove,axis=0).reset_index(drop=True)

In [11]:
data.head()

#### Filling the NaN values with the Median taken from data points of there respective countries

In [12]:
country_list = data.Country.unique()
fill_list = data.columns[3:]#['Life_Expectancy','Adult_Mortality','Alcohol','HepatitisB','BMI','Polio','Tot_Exp','Diphtheria','GDP','Population','thinness_1to19_years','thinness_5to9_years','Income_Comp_Of_Resources','Schooling']

In [13]:
country_list = data.Country.unique()
for feat in fill_list:
    new = []
    i=0
    for i in range(len(data[feat])):
        val = data[feat].iloc[i]
        if pd.isna(val):
            cou = data[data.index==i].Country.values[0]
            mead = data[data.Country == cou][feat].median()
            if pd.isna(mead):
                mead = data[feat].median() 
            data[feat].iloc[i] = mead
           

In [14]:
data.isna().sum()

# OutLiers

In [15]:
col_dict = {data.columns[3+i]:i+1 for i in range(len(data.columns[3:])) }
col_dict

In [16]:
plt.figure(figsize=(20,30))

for variable,i in col_dict.items():
                     plt.subplot(5,4,i)
                     plt.boxplot(data[variable],whis=1.5)
                     plt.title(variable)

plt.show()

#### Let's plot the frap to analyse the outliers in detail

In [17]:
fig,axe = plt.subplots(len(col_dict),2,figsize=(15,80))
c = 0
for variable in col_dict.keys():
    q75, q25 = np.percentile(data[variable], [75 ,25])
    iqr = q75 - q25
    min_val = q25 - (iqr*1.5)
    max_val = q75 + (iqr*1.5)
    lmax = len(data[data[variable]>max_val])
    lmin = len(data[data[variable]<min_val])
    sns.scatterplot(ax=axe[c][0],x=data['Life expectancy '],y=data[variable],color='blue',label='normal')
        
    if lmin!=0:
        sns.scatterplot(ax=axe[c][1],x=data[data[variable]>min_val]['Life expectancy '],y=data[data[variable]>min_val][variable],color='blue',label='normal')
        sns.scatterplot(ax=axe[c][1],x=data[data[variable]<=min_val]['Life expectancy '],y=data[data[variable]<=min_val][variable],color='red',label='outlier')
    if lmax !=0:
        sns.scatterplot(ax=axe[c][1],x=data[data[variable]<max_val]['Life expectancy '],y=data[data[variable]<max_val][variable],color='blue',label='normal')
        sns.scatterplot(ax=axe[c][1],x=data[data[variable]>=max_val]['Life expectancy '],y=data[data[variable]>=max_val][variable],color='red',label='outlier')
    plt.legend()
    c=c+1

In [18]:
d3 = data.copy()

#### Removing Outliers

In [19]:
def fun(x,min_val,max_val):
    if x<min_val:
        return min_val
    elif x>max_val:
        return max_val
    else:
        return x

for variable in col_dict.keys():
    q75, q25 = np.percentile(data[variable], [75 ,25])
    iqr = q75 - q25
    min_val = q25 - (iqr*1.5)
    max_val = q75 + (iqr*1.5)
    data[variable] = data[variable].apply(lambda x:fun(x,min_val,max_val))

# EDA

In [20]:
# Calculating Avg Life Expect.. Per Year
avg_life = [data[data['Year']==year]['Life expectancy '].mean() for year in data.Year.value_counts().keys()]

In [21]:
fig = plt.figure(figsize=(10,5))
sns.scatterplot(list(data.Year.value_counts().keys()),avg_life)
sns.lineplot(list(data.Year.value_counts().keys()),avg_life)
plt.ylabel('Avg Life Expectency')
plt.xlabel('Years')
plt.title('Trend for Life Expectency wrt Year')

#### So we can see a very natural conclusion that with the increase in Medical Science avg lie expectancy also increases

## Relationship of Status with Life Expectancy

In [23]:
sns.barplot(x=data.Status,y=data['Life expectancy '])

## Relationship of Country with Life Expectancy (Latest Life Expectency)

In [24]:
X = list(data.Country.value_counts().keys())#[:20]
avg_life_country = [data[data['Country']==year]['Life expectancy '].max() for year in X]

In [25]:
fig = plt.figure(figsize=(20,8))
sns.barplot(X[:5],y=avg_life_country[:5])

#### Variation in Life Expectancy of Devloped Countries

In [26]:
data[data['Status']=='Developed']['Life expectancy '].var()

#### Variance : 15.45

In [27]:
fig = plt.figure(figsize=(25,10))
sns.barplot(data=data[data['Status']=='Developed'],x='Country',y='Life expectancy ')
plt.title('Variance in Life Expectency of Devloped Countries',fontdict={'fontsize':30})

#### Variation in Life Expectancy of Devloping Countries

In [28]:
data[data['Status']=='Developing']['Life expectancy '].var()

In [29]:
developing = data[data['Status']=='Developing']
fig = plt.figure(figsize=(25,10))
sns.barplot(data=developing,x='Country',y='Life expectancy ')
plt.title('Variance in Life Expectency of Devloping Countries',fontdict={'fontsize':30})

<h3>
From the above graphs It is quite clear that <br>
For <strong> Devloped Countries </strong> the <strong> Variance is quite low </strong>,<br> But
since there is <strong> a large variance </strong> in <strong> Devloping countries <br>
</strong> we cannot Perform Binning on these features.
</h3>

## Relationship between Life Expectancy and Mortality, Infant Death

In [30]:
fig = plt.figure(figsize=(10,5))
sns.lineplot(data=data,x='Year',y='Adult Mortality',label = 'Adult Mortality')
sns.lineplot(data=data,x='Year',y='infant deaths',label = 'Infant Deaths')
sns.lineplot(data=data,x='Year',y='Life expectancy ',label = 'Life expectancy ')
plt.legend()

<strong>It is quite clear that with years Mortality and Infant Deaths Reduces and <br> hence these explaines why Life expectancy increases with time </strong>

In [31]:
data.columns

In [33]:
sns.pairplot(data.iloc[3:])

## Let's see how are the Numerical Columns related with Life Expectancy

In [46]:
ma.FeatureSelection.Draw_Corr_map(data.iloc[:,3:],target_index=0)

### Lets Choose all the Features having Corelation above 0.4

In [47]:
feat = ma.FeatureSelection.Corrilation_selection(data.iloc[:,3:],target_index=0,Minimum_Corr=0.4)

In [96]:
data.iloc[:,3:].shape

### So out of 19 Numerical Features we are have selected 14 Features

In [97]:
# Concatinating Numerical Features with Catagorical Features
data3 = pd.concat([data.iloc[:,:3],feat],axis = 1)

In [98]:
data3.head()

In [99]:
data2 = pd.concat([data.iloc[:,:3],feat],axis = 1)

In [100]:
data2 = ma.handel_standardization(data2)
data2 = ma.handel_Catagorical(data2)

In [101]:
data2.head()

In [109]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

### Spliting the data

In [114]:
Y = data['Life expectancy ']
X_train, X_test, y_train, y_test = train_test_split(dd, Y, test_size=0.30)

## Linear Regression Model

In [125]:
Linear_model= LinearRegression()
Linear_model.fit(X_train,y_train)
pred = Linear_model.predict(X_test)

In [126]:
linReg_on_train = Linear_model.score(X_train,y_train)
linReg_on_test = r2_score(pred,y_test)

## Random Forest

In [128]:
rfc = RandomForestRegressor()
rfc.fit(X_train,y_train)

In [129]:
Train_score_RandForest = rfc.score(X_train,y_train)
Test_score_RandForest = rfc.score(X_test,y_test)

In [135]:
round(Train_score_RandForest*100),round(Test_score_RandForest*100)

In [134]:
round(linReg_on_train*100),round(linReg_on_test*100)

<h1>
    1. Accuracy(approx) on Train data : Random Forest 99, Linear Regresion 96<br>
    2. Accuracy(approx) on Test data : Random Forest 95, Linear Regresion 95

</h1>

## The Above Observation Shows a clear Signs of Overfitting in Random Forest hence we have selected **Linear Regresion Model** as our Final Model